<a href="https://colab.research.google.com/github/Gooogr/YOLO_toolkit/blob/master/2)TrainCustomYOLOweights_pjreddie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 import cv2

In [0]:
# Enable GPU
# Check what we get: K80, P100, K4 or T4.
! nvidia-smi

Mon May 25 14:46:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# mount google disk
from google.colab import drive
drive.mount('/content/gdrive')

# create a symbolic sub-link to avoid 'space' problem
!ln -s /content/gdrive/My\ Drive/colab_notebooks/ /mydrive

#### DarkNet preparations

In [0]:
% cd /mydrive/YOLO_toolkit

/content/gdrive/My Drive/colab_notebooks/YOLO_toolkit


In [0]:
def make_darknet():
  '''
  Run this function if darknet wasn`t setted up your root project folder
  '''
  # Clone and  make darknet in My Drive folder. We should prepare all config files there befory copy them to tmp
  ! git clone https://github.com/pjreddie/darknet

  # Enable GPU learning
  %cd darknet
  !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
  !sed -i 's/GPU=0/GPU=1/' Makefile
  !sed -i 's/CUDNN=0/CUDNN=1/' Makefile

  # verify CUDA
  !/usr/local/cuda/bin/nvcc --version

  ! make

  # Download pre-trainded darknet model weights
  ! wget https://pjreddie.com/media/files/darknet53.conv.74

  # Download pre-trained tiny yolo weights for further extraction
  ! wget https://pjreddie.com/media/files/yolov3-tiny.weights

In [0]:
## Uncomment to make absense darknet from scratch 
# make_darknet()

Also you can edit some code in examples/detector.c. For example change saving weights rules:

Line 139
```
if(i%10000==0 || (i < 1000 && i%100 == 0)){ 
  
if(i%1000==0 || (i < 2000 && i%200 == 0)){ 
```

Don't forget to ```! make``` darknet after that again.

In [0]:
# Create empty directiries for saving weights during the training
! mkdir /mydrive/YOLO_toolkit/yolo_weights


mkdir: cannot create directory ‘/mydrive/YOLO_toolkit/yolo_weights’: File exists


In [0]:
# Create train and test(validate) txt files. Split ratio will be 0.1
% cd /mydrive/YOLO_toolkit/
! python3 splitTrainAndTest.py /mydrive/YOLO_toolkit/Dataset/

/content/gdrive/My Drive/colab_notebooks/YOLO_toolkit


In [0]:
! ls /mydrive/YOLO_toolkit

'1)OpenImagesV4Loader.ipynb'	   splitTrainAndTest.py
'2)TrainCustomYOLOweights.ipynb'   temp_train.log
'Combine datasets.ipynb'	   test_images
 darknet			   test.txt
 darknet_files			   train.txt
 Dataset			   VOC2YOLODatasetConverter.py
 OIDv4_ToolKit			   yolo_weights


Now we have to edit some files. Go to YOLO_toolkit/darknet_files

> .cfg files: yolov3_train.cfg/yolov3_predict.cfg or yolov3tiny_train.cfg/yolov3tiny_predict.cfg
How to configure them:

* Comment/uncomment train/test part for specifying batch and subdivisions parameters
* Minimum *maxbatches* = number_of_your_classes * 2000. Of course you can use a bigger values
* *steps* = *maxbatches* * 0.8, *maxbatches* * 0.9
* *classes* = number_of_your_classes
* *filters* = 3 * (*classes*+5)

> obj.names

Write labels in the same way as in  the classes.txt file. One class - one line. If you make a mistake in the order of the labels, the algorithm will work, but will incorrectly name the detected objects.

> yolo.data

Specify classes amount. Check pathes to other files but they should be right by default:<br>
valid  = /mydrive/YOLO_toolkit/test.txt<br>
names = /mydrive/YOLO_toolkit/obj.names<br>
backup = /mydrive/YOLO_toolkit/yolo_weights<br>
train  = /mydrive/YOLO_toolkit/train.txt<br>

In [0]:
! ls

'1)OpenImagesV4Loader.ipynb'	   splitTrainAndTest.py
'2)TrainCustomYOLOweights.ipynb'   temp_train.log
'Combine datasets.ipynb'	   test_images
 darknet			   test.txt
 darknet_files			   train.txt
 Dataset			   VOC2YOLODatasetConverter.py
 OIDv4_ToolKit			   yolo_weights


In [0]:
! cat darknet_files/obj.names

Human face
Human hand


In [0]:
! cat darknet_files/yolo.data

classes = 2
valid  = /mydrive/YOLO_toolkit/test.txt
names = /mydrive/YOLO_toolkit/darknet_files/obj.names
backup = /mydrive/YOLO_toolkit/yolo_weights
train  = /mydrive/YOLO_toolkit/train.txt



In [0]:
# Get tiny yolo convolutions layers weights
% cd /mydrive/YOLO_toolkit/darknet
! chmod 777 ./darknet
! ./darknet partial /mydrive/YOLO_toolkit/darknet_files/yolov3tiny_train.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

/content/gdrive/My Drive/colab_notebooks/YOLO_toolkit/darknet
layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max        

In [0]:
# Train YOLO
% cd /mydrive/YOLO_toolkit/darknet
! chmod 777 ./darknet
! ./darknet detector train /mydrive/YOLO_toolkit/darknet_files/yolo.data /mydrive/YOLO_toolkit/darknet_files/yolov3tiny_train.cfg ./yolov3-tiny.conv.15 > /mydrive/YOLO_toolkit/temp_train.log


/content/gdrive/My Drive/colab_notebooks/YOLO_toolkit/darknet
layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max        

In [0]:
# Predict
% cd /mydrive/YOLO_toolkit/darknet
! chmod 777 ./darknet
!./darknet detector test /mydrive/YOLO_toolkit/darknet_files/yolo.data /mydrive/YOLO_toolkit/darknet_files/yolov3tiny_predict.cfg /mydrive/YOLO_toolkit/yolo_weights/yolov3tiny_train_final.weights /mydrive/YOLO_toolkit/test_images/crowd.jpg

/content/gdrive/My Drive/colab_notebooks/YOLO_toolkit/darknet
layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max        

In [0]:
# def imShow(path):
#   image = cv2.imread(path)
#   height, width = image.shape[:2]
#   resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

#   cv2.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))